# Merge DataFrames in Pandas

This notebooks shows how to read Pandas DataFrames from an Excel workbook and merge them based on specific rows. 


In [1]:
import json
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
EXCEL_SOURCE_FILE = "example_workbook.xlsx"

First, the interface data from the Excel sheet is loaded to a new pandas DataFrame. To verify that the DataFrame contains some data, the `head()` function is used. It shows the first five entries within the DataFrame.

Additional information about the `read_excel()` method is available in the [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html).

In [3]:
interface_df = pd.read_excel(EXCEL_SOURCE_FILE, sheetname="interface")
interface_df.head()

,hostname,interface_name,description,port_role
0,Switch_A,g0/1,Router Port,Router
1,Switch_A,g0/2,NaN,Data_Port
2,Switch_A,g0/3,NaN,Data_Port
3,Switch_A,g0/4,NaN,Data_Port
4,Switch_A,g0/5,NaN,Data_Port


The DataFrame should now be extended with information from the `port_role` sheet. 

In [4]:
port_role_df = pd.read_excel(EXCEL_SOURCE_FILE, sheetname="port_role")
port_role_df.head()

,name,Access VLAN ID,Voice VLAN ID,Trunk VLAN List
0,Data_Port,10,NaN,NaN
1,Voice_Port,10,20.0,NaN
2,Switch_to_Switch,1,NaN,1-100
3,Router,80,NaN,NaN
4,Printer_Port,11,NaN,NaN


Pandas provides a nice feature to merge data from two DataFrames by a specific column name. We have different key names in this example, therefore we need to specify the `left_on` and `right_on` parameter to identify the column that should be used for the merge operation.

**Please Note:** The merge operation will create a new DataFrame that contains the values of the both DataFrames. The existing one won't be changed

Further information about the merge function is available at [pandas documentation on merge](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)

In [5]:
full_intf_df = pd.merge(interface_df, port_role_df, left_on="port_role", right_on="name")
full_intf_df.head(10)

,hostname,interface_name,description,port_role,name,Access VLAN ID,Voice VLAN ID,Trunk VLAN List
0,Switch_A,g0/1,Router Port,Router,Router,80,NaN,NaN
1,Switch_A,g0/2,NaN,Data_Port,Data_Port,10,NaN,NaN
2,Switch_A,g0/3,NaN,Data_Port,Data_Port,10,NaN,NaN
3,Switch_A,g0/4,NaN,Data_Port,Data_Port,10,NaN,NaN
4,Switch_A,g0/5,NaN,Data_Port,Data_Port,10,NaN,NaN
5,Switch_A,g0/6,NaN,Voice_Port,Voice_Port,10,20.0,NaN
6,Switch_A,g0/7,NaN,Voice_Port,Voice_Port,10,20.0,NaN
7,Switch_A,g0/8,NaN,Voice_Port,Voice_Port,10,20.0,NaN
8,Switch_A,g0/9,NaN,Voice_Port,Voice_Port,10,20.0,NaN
9,Switch_A,g0/10,NaN,Voice_Port,Voice_Port,10,20.0,NaN


The new DataFrame contains now the general interface information and the information from the `port_role` sheet. The `port_role` and `name` contains the same values, therefore one can be removed from the DataFrame.

In [6]:
del full_intf_df["name"]
full_intf_df.head(5)

,hostname,interface_name,description,port_role,Access VLAN ID,Voice VLAN ID,Trunk VLAN List
0,Switch_A,g0/1,Router Port,Router,80,NaN,NaN
1,Switch_A,g0/2,NaN,Data_Port,10,NaN,NaN
2,Switch_A,g0/3,NaN,Data_Port,10,NaN,NaN
3,Switch_A,g0/4,NaN,Data_Port,10,NaN,NaN
4,Switch_A,g0/5,NaN,Data_Port,10,NaN,NaN


For the next steps, only the interface information for `Switch_A` are used.

In [7]:
switch_a_intf_df = full_intf_df[full_intf_df.hostname == "Switch_A"]
switch_a_intf_df

,hostname,interface_name,description,port_role,Access VLAN ID,Voice VLAN ID,Trunk VLAN List
0,Switch_A,g0/1,Router Port,Router,80,NaN,NaN
1,Switch_A,g0/2,NaN,Data_Port,10,NaN,NaN
2,Switch_A,g0/3,NaN,Data_Port,10,NaN,NaN
3,Switch_A,g0/4,NaN,Data_Port,10,NaN,NaN
4,Switch_A,g0/5,NaN,Data_Port,10,NaN,NaN
5,Switch_A,g0/6,NaN,Voice_Port,10,20.0,NaN
6,Switch_A,g0/7,NaN,Voice_Port,10,20.0,NaN
7,Switch_A,g0/8,NaN,Voice_Port,10,20.0,NaN
8,Switch_A,g0/9,NaN,Voice_Port,10,20.0,NaN
9,Switch_A,g0/10,NaN,Voice_Port,10,20.0,NaN


The hostname is removed, because it's not required anymore.

In [8]:
del switch_a_intf_df["hostname"]
switch_a_intf_df

,interface_name,description,port_role,Access VLAN ID,Voice VLAN ID,Trunk VLAN List
0,g0/1,Router Port,Router,80,NaN,NaN
1,g0/2,NaN,Data_Port,10,NaN,NaN
2,g0/3,NaN,Data_Port,10,NaN,NaN
3,g0/4,NaN,Data_Port,10,NaN,NaN
4,g0/5,NaN,Data_Port,10,NaN,NaN
5,g0/6,NaN,Voice_Port,10,20.0,NaN
6,g0/7,NaN,Voice_Port,10,20.0,NaN
7,g0/8,NaN,Voice_Port,10,20.0,NaN
8,g0/9,NaN,Voice_Port,10,20.0,NaN
9,g0/10,NaN,Voice_Port,10,20.0,NaN


The column names are directly used from the Excel sheet. To use them, for example in configuration templates, we need to clean the column names (remove the blanks and make them uppercase). The following code renames all columns.

In [9]:
# we will use a list comprehension fot this
column_replacements = dict(
    zip(
        switch_a_intf_df.columns,                                         # the current column names
        [e.upper().replace(" ", "_") for e in switch_a_intf_df.columns]   # the new column names
    )
)
switch_a_intf_df = switch_a_intf_df.rename(columns=column_replacements)
switch_a_intf_df

,INTERFACE_NAME,DESCRIPTION,PORT_ROLE,ACCESS_VLAN_ID,VOICE_VLAN_ID,TRUNK_VLAN_LIST
0,g0/1,Router Port,Router,80,NaN,NaN
1,g0/2,NaN,Data_Port,10,NaN,NaN
2,g0/3,NaN,Data_Port,10,NaN,NaN
3,g0/4,NaN,Data_Port,10,NaN,NaN
4,g0/5,NaN,Data_Port,10,NaN,NaN
5,g0/6,NaN,Voice_Port,10,20.0,NaN
6,g0/7,NaN,Voice_Port,10,20.0,NaN
7,g0/8,NaN,Voice_Port,10,20.0,NaN
8,g0/9,NaN,Voice_Port,10,20.0,NaN
9,g0/10,NaN,Voice_Port,10,20.0,NaN


The `NaN` values are present if the DataFrame is converted to a string representation. To replace these values with an empty string, use the following `fillna()` function.

In [10]:
clean_list = switch_a_intf_df.fillna("")
clean_list

,INTERFACE_NAME,DESCRIPTION,PORT_ROLE,ACCESS_VLAN_ID,VOICE_VLAN_ID,TRUNK_VLAN_LIST
0,g0/1,Router Port,Router,80,,
1,g0/2,,Data_Port,10,,
2,g0/3,,Data_Port,10,,
3,g0/4,,Data_Port,10,,
4,g0/5,,Data_Port,10,,
5,g0/6,,Voice_Port,10,20,
6,g0/7,,Voice_Port,10,20,
7,g0/8,,Voice_Port,10,20,
8,g0/9,,Voice_Port,10,20,
9,g0/10,,Voice_Port,10,20,


Now, every row is converted to a `dictionary` and added to another dictionary that can be used, e.g. with Jinja2 to generate a configurations. I wrote another article about the [configuration generation with python and Jinja2](https://codingnetworker.com/2015/09/configuration-generator-with-python-and-jinja2/).

In [11]:
result = {
    "HOSTNAME": "Switch A",  # we only have Switch A in this case
    "PORTS": []
}
for index, row in clean_list.iterrows():
    result["PORTS"].append(row.to_dict())

print(json.dumps(result, indent=4))

{
    "HOSTNAME": "Switch A",
    "PORTS": [
        {
            "DESCRIPTION": "Router Port",
            "ACCESS_VLAN_ID": 80,
            "INTERFACE_NAME": "g0/1",
            "TRUNK_VLAN_LIST": "",
            "VOICE_VLAN_ID": "",
            "PORT_ROLE": "Router"
        },
        {
            "DESCRIPTION": "",
            "ACCESS_VLAN_ID": 10,
            "INTERFACE_NAME": "g0/2",
            "TRUNK_VLAN_LIST": "",
            "VOICE_VLAN_ID": "",
            "PORT_ROLE": "Data_Port"
        },
        {
            "DESCRIPTION": "",
            "ACCESS_VLAN_ID": 10,
            "INTERFACE_NAME": "g0/3",
            "TRUNK_VLAN_LIST": "",
            "VOICE_VLAN_ID": "",
            "PORT_ROLE": "Data_Port"
        },
        {
            "DESCRIPTION": "",
            "ACCESS_VLAN_ID": 10,
            "INTERFACE_NAME": "g0/4",
            "TRUNK_VLAN_LIST": "",
            "VOICE_VLAN_ID": "",
            "PORT_ROLE": "Data_Port"
        },
        {
            "DESCRIPT